<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/4_analysis_hate_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Analisis del Dataset completo marzo 2020 - abril 2021 de X

**Natalia Dedandi**

Se aplica el clasificador seleccionado a todo el conjunto de datos recoelctado para analizar las características del odio en Argentina durante la pandemia

In [1]:
!pip install datasets seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires

Recupero el conjutno completo de datos: https://huggingface.co/datasets/piuba-bigdata/articles_and_comments



In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("piubamas/articles_and_comments")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/537201 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
        num_rows: 537201
    })
})

In [4]:
ds1 = dataset["train"]

Ejemplo de una fila

In [5]:
ds1[123]

{'tweet_id': '1376979522449473540',
 'text': 'Boca le gana a River en el ránking de mejores equipos del siglo XXI https://t.co/uwVGKDyuuy',
 'title': None,
 'url': None,
 'user': 'pagina12',
 'body': None,
 'created_at': '2021-03-30T19:27:55Z',
 'comments': [{'created_at': '2021-03-30T19:30:36Z',
   'prediction': {'APPEARANCE': 0,
    'CALLS': 0,
    'CLASS': 0,
    'CRIMINAL': 0,
    'DISABLED': 0,
    'LGBTI': 0,
    'POLITICS': 0,
    'RACISM': 0,
    'WOMEN': 0},
   'text': '@pagina12 A quien mierda le importa eso ? Boca murió en Madrid. Fin de la historia.',
   'tweet_id': '1376980196226633731',
   'user_id': '2707560671'},
  {'created_at': '2021-03-30T19:33:50Z',
   'prediction': {'APPEARANCE': 0,
    'CALLS': 0,
    'CLASS': 0,
    'CRIMINAL': 0,
    'DISABLED': 0,
    'LGBTI': 0,
    'POLITICS': 0,
    'RACISM': 0,
    'WOMEN': 0},
   'text': '@pagina12 Y algo tenía que ganar!😂😂😂😂',
   'tweet_id': '1376981010626265091',
   'user_id': '1171698223209205760'},
  {'created_at': '20

En este conjunto de datos tengo las etiquetas de la clasificación realizada con el modelo BETO:

https://huggingface.co/piuba-bigdata/beto-contextualized-hate-speech

Puede usarse para comparar pero en principio nos interesa recuperar los comentarios completos para aplicarle el clasificador seleccionado